In [ ]:
!pip install rasterio
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Set Folder Path in Google Drive
folder_path = "/content/drive/MyDrive/mapbiomasgdalubuntu"

# List GeoTIFF files in folder
tif_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.tif') and f.startswith('mapbiomas_')])
#tif_files = [f for f in os.listdir(folder_path) if f.endswith('.tif') and f.startswith('mapbiomas_')]

# Make a raster for natural classes
for file in tif_files:
    year = file.split('_')[-1].split('.')[0]  # Extract file name
    file_path = os.path.join(folder_path, file)

    # Open original raster
    with rasterio.open(file_path) as src:
        profile = src.profile  # Catch original raster Metadata
        data = src.read(1)  # Read the first band

        # Make a binary natural classes mask (3, 4, 11, 12)
        natureza_bin = np.where(np.isin(data, [3, 4, 11, 12]), 1, 0).astype(np.uint8)

        # Update profile ti save in uint8 (values 0 ou 1)
        profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

        # Save this layer
        natureza_output_path = os.path.join(folder_path, f"natureza_{year}.tif")
        with rasterio.open(natureza_output_path, 'w', **profile) as dst:
            dst.write(natureza_bin, 1)

    print(f"Processado: {file} -> natureza_{year}.tif")

print("Processamento concluído!")

In [ ]:
#Deforestation age
import rasterio
import numpy as np
import os
import pandas as pd
from rasterio.enums import Resampling
import matplotlib.pyplot as plt
from rasterio.transform import xy

# Path files
raster_folder = "/content/drive/MyDrive/mapbiomasgdalubuntu/"
natural_vegetation_files = sorted([os.path.join(raster_folder, f) for f in os.listdir(raster_folder) if f.startswith("natureza_") and f.endswith(".tif")])
output_deforestation_age_path = os.path.join(raster_folder, "deforestation_age_colab.tif")

# Set range of years
anos = list(range(1985, 2024))  # 1985 a 2023
num_anos = len(anos)

# Open the first file to set metadata
with rasterio.open(natural_vegetation_files[0]) as src:
    meta = src.meta.copy()
    altura, largura = src.shape

# Make an array to allocate deforestation data
idade_desmate = np.zeros((altura, largura), dtype=np.uint8)

# Start values in 39 
with rasterio.open(natural_vegetation_files[0]) as src:
    img_anterior = src.read(1)
    idade_desmate[img_anterior == 0] = 39

# Run temporal series (1985 to 2023) 
for i in range(1, num_anos):
    ano_atual = anos[i]
    idade_atual = num_anos - i 

    with rasterio.open(natural_vegetation_files[i]) as src:
        img_atual = src.read(1)

        # Identify change from 1 to 0
        desmatado = (img_anterior == 1) & (img_atual == 0) & (idade_desmate == 0)
        idade_desmate[desmatado] = idade_atual

        # Update previous image for next iteration
        img_anterior = img_atual.copy()

    print(f"Processado: {ano_atual}, pixels de desmatamento identificados: {np.sum(desmatado)}")

# Update metadata
meta.update({
    "dtype": 'uint8',
    "count": 1,
    "nodata": 0
})

# Update metadata profile for the output deforestation file
with rasterio.open(output_deforestation_age_path, "w", **meta) as dst:
    dst.write(idade_desmate, 1)

# If you want see your rasters
plt.figure(figsize=(10, 6))
plt.imshow(idade_desmate, cmap='viridis')
plt.colorbar(label="Idade do desmatamento (anos)")
plt.title("Primeiro ano de desmatamento detectado")
plt.show()

print("Raster de idade do desmatamento criado com sucesso!")
